<br>
<h2 style = "font-size:40px; font-family:Garamond ; font-weight : normal; background-color: #007580; color :#fed049   ; text-align: center; border-radius: 5px 5px; padding: 5px"> Amazon Electronics - Recommender System</h2> 
<br>

<p style = "font-size:30px; color: #007580 ;background-color:  ; text-align: left; border-radius: 5px 5px; padding: 5px" ><strong> Here are my other notebooks, please have a look and definitely you will find it useful. Happy reading 🙂</strong></p>
<ol>
<li><a href ="https://www.kaggle.com/vinayakshanawad/industrial-safety-complete-solution">Industrial Safety - Complete Solution</a></li>
<li><a href ="https://www.kaggle.com/vinayakshanawad/eda-statistical-analysis-hypothesis-testing">EDA - Statistical Analysis - Hypothesis Testing</a></li>
<li><a href ="https://www.kaggle.com/vinayakshanawad/random-forest-with-bootstrap-sampling-for-beginner">Random Forest with Bootstrap Sampling for beginner</a></li>
</ol>

<a id = '0'></a>
<h2 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #007580; color : #fed049; border-radius: 5px 5px; text-align:center; font-weight: bold" >Table of Contents</h2> 

1. [Overview](#1.0)
2. [Import the necessary libraries](#2.0)
3. [Data Collection](#3.0)
4. [Data Cleaning](#4.0)
	- [4.1 Check Duplicates](#4.1)
	- [4.2 Check Missing Values](#4.2)
	- [4.3 Check Outliers](#4.3)
	- [4.4 Data Cleaning Summary](#4.4)
5. [EDA (Data Analysis and Preparation)](#5.0)
	- [5.1 Check the distribution of ratings - Ratings are on scale of 1-5](#5.1)
	- [5.2 Take a subset of the dataset to make it less sparse/ denser](#5.2)
	- [5.3 Calculate the density of the rating matrix](#5.3)
6. [Model Building and Validation](#6.0)
	- [6.1 Sampling Techniques - Create Training and Test Set](#6.1)
	- [6.2 Build Popularity Recommender model](#6.2)
	- [6.3 Build User Based Collaborative Filtering model](#6.3)
	- [6.4 SVD Based Recommendation System using scipy library](#6.4)
    - [6.5 Evaluate the Collaborative recommender model](#6.5)
    - [6.6 Getting top - K ( K = 5) recommendations](#6.6)
    - [6.7 SVD Based Recommendation System using Surprise library](#6.7)
    - [6.8 KNNWithMeans](#6.8)
    - [6.9 Generating top n recommendations](#6.9)
7. [Conclusion](#7.0)

<a id = '1.0'></a>
<h2 style = "font-size:35px; font-family:Garamond ; font-weight : normal; background-color: #007580; color :#fed049   ; text-align: center; border-radius: 5px 5px; padding: 5px"> 1. Overview </h2>

### Domain: E-commerce

### Context:

Everyday a million products are being recommended to users based on
popularity and other metrics on e-commerce websites. The most popular e-commerce
website boosts average order value by 50%, increases revenues by 300%, and
improves conversion. In addition to being a powerful tool for increasing revenues,
product recommendations are so essential that customers now expect to see similar
features on all other eCommerce sites.

### Data Description:

Data columns- First three columns are userId, productId, and ratings and the fourth
column is timestamp. You can discard the timestamp column as in this case you may
not need to use it.

### Source:

Amazon Reviews data (http://jmcauley.ucsd.edu/data/amazon/) The
repository has several datasets. For this case study, we are using the Electronics
dataset.

### Objective:

**To make a recommendation system that recommends at least five(5) new products based on the user's habits.**

<a id = '2.0'></a>
<h2 style = "font-size:35px; font-family:Garamond ; font-weight : normal; background-color: #007580; color :#fed049   ; text-align: center; border-radius: 5px 5px; padding: 5px"> 2. Import the necessary libraries </h2> 

In [ ]:
pip install surprise

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

# used to supress display of warnings
import warnings

import missingno as mno

from sklearn.model_selection import train_test_split
from scipy.sparse.linalg import svds

from surprise import Dataset,Reader
from surprise import SVD, KNNWithMeans
from surprise import accuracy

#### Setting Options

In [ ]:
# suppress display of warnings
warnings.filterwarnings('ignore')

# display all dataframe columns
pd.options.display.max_columns = None

# to set the limit to 3 decimals
pd.options.display.float_format = '{:.7f}'.format

# display all dataframe rows
pd.options.display.max_rows = None

<a id = '3.0'></a>
<h2 style = "font-size:35px; font-family:Garamond ; font-weight : normal; background-color: #007580; color :#fed049   ; text-align: center; border-radius: 5px 5px; padding: 5px"> 3. Data Collection </h2>

In [ ]:
# Reading Amazon electronics ratings data
ele_ratings_df = pd.read_csv("../input/amazon-electronic-product-recommendation/ratings_Electronics (1).csv")

In [ ]:
# Get the top 5 rows
ele_ratings_df.head()

<p style = "font-size:20px; color: #007580 "><strong> Shape of the data </strong></p> 

In [ ]:
# Get the shape of electronics ratings data
ele_ratings_df.shape

In [ ]:
print("Number of rows = {0} and Number of Columns = {1} in Data frame".format(ele_ratings_df.shape[0],ele_ratings_df.shape[1]))

* Since the data is very big. Consider ele_ratings_df named dataframe with  first 10% of data which contains 782448 rows and all columns from 0 of dataset.

In [ ]:
# Taking 10% of the dataset
ele_ratings_df = ele_ratings_df.iloc[:782448,0:]

<p style = "font-size:20px; color: #007580 "><strong> Data type of each attribute </strong></p>

In [ ]:
# Check datatypes
ele_ratings_df.dtypes

**As mentioned in data description, frst three columns are userId, productId, and ratings and the fourth column is timestamp. We can discard the timestamp column as in this case you may not need to use it.**

<a id = '4.0'></a>
<h2 style = "font-size:35px; font-family:Garamond ; font-weight : normal; background-color: #007580; color :#fed049   ; text-align: center; border-radius: 5px 5px; padding: 5px"> 4. Data Cleaning </h2>

<p style = "font-size:20px; color: #007580 "><strong> Rename the columns </strong></p>

In [ ]:
ele_ratings_df.rename(columns = {'AKM1MP6P0OYPR':'userId', '0132793040':'productId', '5.0':'ratings', 
                                 '1365811200':'timestamp',}, inplace = True)

<p style = "font-size:20px; color: #007580 "><strong> Remove timestamp column as in this case it is not required for analysis </strong></p>

In [ ]:
# Remove timestamp column
ele_ratings_df.drop(['timestamp'], axis = 1, inplace = True)

In [ ]:
# Check electronics ratings info
ele_ratings_df.info()

In [ ]:
# Get the top 5 rows
ele_ratings_df.head()

<a id = '4.1'></a>
<p style = "font-size:20px; color: #007580 "><strong> 4.1 Check Duplicates </strong></p>

In [ ]:
# Check Duplicates
ele_ratings_df.duplicated().sum()

<a id = '4.2'></a>
<p style = "font-size:20px; color: #007580 "><strong> 4.2 Check Missing Values </strong></p>

In [ ]:
# Check the presence of missing values
ele_ratings_df.isnull().sum()

In [ ]:
# Visualize missing values
mno.matrix(ele_ratings_df, figsize = (10, 5));

<a id = '4.3'></a>
<p style = "font-size:20px; color: #007580 "><strong> 4.3 Check Outliers </strong></p>

As we know, cold start problems can be outliers in recommender systems which means neither there is a new user who has not purchased any of the products or not rated any products nor there is a new product which is not purchased by any user or not received any ratings.

In [ ]:
# Check outliers 
np.where(ele_ratings_df['ratings'] < 1)

In [ ]:
# Check outliers
ele_ratings_df['ratings'].unique()

<a id = '4.4'></a>
<p style = "font-size:20px; color: #007580 "><strong> 4.4 Data Cleaning Summary </strong></p>

1. No duplicates in this dataset.
2. No outliers (cold start problems) in this dataset.
3. No missing values in this dataset.
4. Dropped timestamp column as this is not required for analysis.

<a id = '5.0'></a>
<h2 style = "font-size:35px; font-family:Garamond ; font-weight : normal; background-color: #007580; color :#fed049   ; text-align: center; border-radius: 5px 5px; padding: 5px"> 5. EDA (Data Analysis and Preparation) </h2>

In [ ]:
# Summary statistics of 'rating' variable
ele_ratings_df.describe()

<a id = '5.1'></a>
<p style = "font-size:20px; color: #007580 "><strong> 5.1 Check the distribution of ratings - Ratings are on scale of 1-5 </strong></p>

In [ ]:
# Check the distribution of ratings 
with sns.axes_style('white'):
    g = sns.factorplot("ratings", data=ele_ratings_df, aspect=2.0, kind='count');
    g.set_ylabels("Total number of ratings") 

In [ ]:
# Count users based on ratings
for i in range(1,6):
  print('Number of users who rated {0} ratings = {1}'.format(i, ele_ratings_df[ele_ratings_df['ratings'] == i].shape[0]))

In [ ]:
# Number of unique user id and product id in the data
print('Number of unique USERS in Raw data = ', ele_ratings_df['userId'].nunique())
print('Number of unique PRODUCTS in Raw data = ', ele_ratings_df['productId'].nunique())

In [ ]:
# Top 10 users based on rating
most_rated_users = ele_ratings_df.groupby('userId').size().sort_values(ascending=False)[:10]
most_rated_users

In [ ]:
# Top 10 products based on rating
most_rated_products = ele_ratings_df.groupby('productId').size().sort_values(ascending=False)[:10]
most_rated_products

<a id = '5.2'></a>
<p style = "font-size:20px; color: #007580 "><strong> 5.2 Take a subset of the dataset to make it less sparse/ denser </strong></p>
<p style = "font-size:20px; color: #007580 "><strong> For example, keep the users only who has given 50 or more number of ratings </strong></p>

In [ ]:
users_count = ele_ratings_df.userId.value_counts()
ele_ratings_df_sample = ele_ratings_df[ele_ratings_df.userId.isin(users_count[users_count >= 50].index)]
print('Number of users who has given 50 or more number of ratings =', len(ele_ratings_df_sample))

In [ ]:
# Number of unique user id and product id in the sample dataset
print('Number of unique USERS in sample dataset = ', ele_ratings_df_sample['userId'].nunique())
print('Number of unique PRODUCTS in sample dataset = ', ele_ratings_df_sample['productId'].nunique())

<a id = '5.3'></a>
<p style = "font-size:20px; color: #007580 "><strong> 5.3 Calculate the density of the rating matrix </strong></p>

In [ ]:
final_ratings_matrix = ele_ratings_df_sample.pivot(index = 'userId', columns ='productId', values = 'ratings')
final_ratings_matrix.head()

In [ ]:
#final_ratings_matrix = final_ratings_matrix.fillna(final_ratings_matrix.mean(axis=0))
final_ratings_matrix = final_ratings_matrix.fillna(0)
print('Shape of final_ratings_matrix: ', final_ratings_matrix.shape)

given_num_of_ratings = np.count_nonzero(final_ratings_matrix)
print('given_num_of_ratings = ', given_num_of_ratings)

possible_num_of_ratings = final_ratings_matrix.shape[0] * final_ratings_matrix.shape[1]
print('possible_num_of_ratings = ', possible_num_of_ratings)

density = (given_num_of_ratings/possible_num_of_ratings)
density *= 100
print ('density: {:4.2f}%'.format(density))

In [ ]:
# Matrix with one row per 'User' and one column per 'ProductId' for User-based CF
final_ratings_matrix.head()

In [ ]:
# Matrix with one row per 'Product' and one column per 'User' for Item-based CF
final_ratings_matrix_T = final_ratings_matrix.transpose()
final_ratings_matrix_T.head()

<a id = '6.0'></a>
<h2 style = "font-size:35px; font-family:Garamond ; font-weight : normal; background-color: #007580; color :#fed049   ; text-align: center; border-radius: 5px 5px; padding: 5px"> 6. Model Building and Validation </h2> 

<a id = '6.1'></a>
<p style = "font-size:20px; color: #007580 "><strong> 6.1 Sampling Techniques - Create Training and Test Set </strong></p> 

In [ ]:
# Split the data randomnly into train and test datasets into 70:30 ratio
train_data, test_data = train_test_split(ele_ratings_df_sample, test_size = 0.3, random_state=123)
train_data.head()

In [ ]:
print('Shape of training data: ',train_data.shape)
print('Shape of testing data: ',test_data.shape)

<a id = '6.2'></a>
<p style = "font-size:20px; color: #007580 "><strong> 6.2 Build Popularity Recommender model </strong></p> 

In [ ]:
# Count of userId for each unique product as recommendation score 
train_data_grouped = train_data.groupby('productId').agg({'userId': 'count'}).reset_index()
train_data_grouped.rename(columns = {'userId': 'score'}, inplace=True)
train_data_grouped.head(10)

In [ ]:
# Sort the products on recommendation score 
train_data_sort = train_data_grouped.sort_values(['score', 'productId'], ascending = [0,1]) 
      
# Generate a recommendation rank based upon score 
train_data_sort['rank'] = train_data_sort['score'].rank(ascending=0, method='first') 
          
# Get the top 5 recommendations 
popularity_recommendations = train_data_sort.head() 
popularity_recommendations 

In [ ]:
# Use popularity based recommender model to make predictions
def recommend(user_id):     
    user_recommendations = popularity_recommendations 
          
    #Add user_id column for which the recommendations are being generated 
    user_recommendations['userId'] = user_id 
      
    #Bring user_id column to the front 
    cols = user_recommendations.columns.tolist() 
    cols = cols[-1:] + cols[:-1] 
    user_recommendations = user_recommendations[cols] 
          
    return user_recommendations 

In [ ]:
find_recom = [10,100,150]   # This list is user choice.
for i in find_recom:
    print("The list of recommendations for the userId: %d\n" %(i))
    print(recommend(i))    
    print("\n") 

**Since, it is a Popularity recommender model, so all the three users are given the same recommendations. Here, we predict the products based on the popularity. It is not personalized to particular user. It is a non-personalized recommender system.**

<a id = '6.3'></a>
<p style = "font-size:20px; color: #007580 "><strong> 6.3 Build User Based Collaborative Filtering model </strong></p> 

In [ ]:
# Matrix with one row per 'User' and one column per 'ProductId' for User-based CF
final_ratings_matrix.head()

In [ ]:
# Define user index from 0 to 10
final_ratings_matrix['user_index'] = np.arange(0, final_ratings_matrix.shape[0], 1)
final_ratings_matrix.head()

In [ ]:
final_ratings_matrix.set_index(['user_index'], inplace=True)

# Actual ratings given by users
final_ratings_matrix.head()

**As this is a sparse matrix, SVD is best to apply on large sparse matrix.**

<a id = '6.4'></a>
<p style = "font-size:20px; color: #007580 "><strong> 6.4 SVD Based Recommendation System using scipy library </strong></p> 

In [ ]:
# Singular Value Decomposition
U, sigma, Vt = svds(final_ratings_matrix, k = 10)

In [ ]:
print('Left singular vector matrix: \n',U)

In [ ]:
print('Sigma: \n',sigma)

**As sigma is not a diagonal matrix we have to convert it into diagonal matrix.**

In [ ]:
# Construct a diagonal matrix in SVD
sigma = np.diag(sigma)
print('Diagonal matrix: \n',sigma)

In [ ]:
print('Right singular vector matrix: \n',Vt)

In [ ]:
all_users_predicted_ratings = np.dot(np.dot(U, sigma), Vt) 

# Predicted ratings
ele_preds_ratings_df = pd.DataFrame(all_users_predicted_ratings, columns = final_ratings_matrix.columns)
ele_preds_ratings_df.head()

In [ ]:
# Recommend the items with the highest predicted ratings
def recommend_items(userId, final_ratings_matrix, ele_preds_ratings_df, num_recommendations):

    # Index starts at 0
    user_idx = userId-1
    
    # Get and sort the user's ratings
    sorted_user_ratings = final_ratings_matrix.iloc[user_idx].sort_values(ascending=False)

    # Sorted_user_ratings
    sorted_user_predictions = ele_preds_ratings_df.iloc[user_idx].sort_values(ascending=False)

    # Sorted_user_predictions
    temp = pd.concat([sorted_user_ratings, sorted_user_predictions], axis=1)
    temp.index.name = 'Recommended Items'
    temp.columns = ['user_ratings', 'user_predictions']
    temp = temp.loc[temp.user_ratings == 0]
    temp = temp.sort_values('user_predictions', ascending=False)
    
    print('\nBelow are the recommended items for user(user_id = {}):\n'.format(userId))
    print(temp.head(num_recommendations))

In [ ]:
# Enter the 'userId' and 'num_recommendations' for the user 5
userId = 5
num_recommendations = 5
recommend_items(userId, final_ratings_matrix, ele_preds_ratings_df, num_recommendations)

In [ ]:
# Enter the 'userId' and 'num_recommendations' for the user 20
userId = 20
num_recommendations = 5
recommend_items(userId, final_ratings_matrix, ele_preds_ratings_df, num_recommendations)

In [ ]:
# Enter the 'userId' and 'num_recommendations' for the user 25
userId = 25
num_recommendations = 5
recommend_items(userId, final_ratings_matrix, ele_preds_ratings_df, num_recommendations)

* Since, it is a Collaborative recommender model, so all the three users are given different recommendations based on users past behaviour.

<a id = '6.5'></a>
<p style = "font-size:20px; color: #007580 "><strong> 6.5 Evaluate the Collaborative recommender model </strong></p> 

(Once the model is trained on the training data, it
can be used to compute the error (like RMSE) on predictions made on the test
data.) You can also use a different method to evaluate the models.

In [ ]:
# Actual ratings given by the users
final_ratings_matrix.head()

In [ ]:
# Average ACTUAL rating for each item
final_ratings_matrix.mean().head()

In [ ]:
# Predicted ratings 
ele_preds_ratings_df.head()

In [ ]:
# Average PREDICTED rating for each item
ele_preds_ratings_df.mean().head()

In [ ]:
rmse_df = pd.concat([final_ratings_matrix.mean(), ele_preds_ratings_df.mean()], axis=1)
rmse_df.columns = ['Avg_actual_ratings', 'Avg_predicted_ratings']
print(rmse_df.shape)
rmse_df['item_index'] = np.arange(0, rmse_df.shape[0], 1)
rmse_df.head()

In [ ]:
RMSE = round((((rmse_df.Avg_actual_ratings - rmse_df.Avg_predicted_ratings) ** 2).mean() ** 0.5), 10)
print('\nRMSE of SVD Model = {} \n'.format(RMSE))

<a id = '6.6'></a>
<p style = "font-size:20px; color: #007580 "><strong> 6.6 Getting top - K ( K = 5) recommendations </strong></p> 

In [ ]:
# Enter the 'userId' and 'num_recommendations' for the user 5
userId = 5
num_recommendations = 5
recommend_items(userId, final_ratings_matrix, ele_preds_ratings_df, num_recommendations)

In [ ]:
# Enter the 'userId' and 'num_recommendations' for the user 20
userId = 20
num_recommendations = 5
recommend_items(userId, final_ratings_matrix, ele_preds_ratings_df, num_recommendations)

In [ ]:
# Enter the 'userId' and 'num_recommendations' for the user 25
userId = 25
num_recommendations = 5
recommend_items(userId, final_ratings_matrix, ele_preds_ratings_df, num_recommendations)

<a id = '6.7'></a>
<p style = "font-size:20px; color: #007580 "><strong> 6.7 SVD Based Recommendation System using Surprise library </strong></p> 

In [ ]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ele_ratings_df_sample[['userId', 'productId', 'ratings']], reader)

In [ ]:
# Split data into train and test
from surprise.model_selection import train_test_split
trainset, testset = train_test_split(data, test_size=.30, random_state=123)

In [ ]:
# Fit the model on Training set
svd_model = SVD(n_factors=10, biased=False)
svd_model.fit(trainset)

In [ ]:
# Evaluation of the model or Validate Training model on Test set

# Predict on Test set
svd_test_pred = svd_model.test(testset)

In [ ]:
# Compute RMSE
accuracy.rmse(svd_test_pred)

<a id = '6.8'></a>
<p style = "font-size:20px; color: #007580 "><strong> 6.8 KNNWithMeans </strong></p> 

In [ ]:
# Fit the model on Training set
knn_model = KNNWithMeans(k=10, sim_options={'user_based': True})
knn_model.fit(trainset)

In [ ]:
# Evaluation of the model or Validate Training model on Test set

# Predict on Test set
knn_test_pred = knn_model.test(testset)

# Compute RMSE
accuracy.rmse(knn_test_pred)

* **KNNWithMeans gave better results compared to SVD models**

* We can try Cross validation and improve accuracy

In [ ]:
uid = "A11D1KHM7DVOQK"  # raw user id (as in the ratings file). They are **strings**!
iid = "B000001OMN"  # raw item id (as in the ratings file). They are **strings**!

# get a prediction for specific users and items.
pred = knn_model.predict(uid, iid, r_ui=0.0, verbose=True)

<a id = '6.9'></a>
<p style = "font-size:20px; color: #007580 "><strong> 6.9 Generating top n recommendations </strong></p> 

In [ ]:
pred = pd.DataFrame(knn_test_pred)
pred[pred['uid'] == "A11D1KHM7DVOQK"][['iid', 'r_ui','est']].sort_values(by = 'r_ui', ascending = False).head(10)

<a id = '7.0'></a>
<h2 style = "font-size:35px; font-family:Garamond ; font-weight : normal; background-color: #007580; color :#fed049   ; text-align: center; border-radius: 5px 5px; padding: 5px"> 7. Conclusion </h2> 

1. The Popularity-based recommender system is a non-personalised recommender system and these are based on frequecy counts, which may be not suitable to the user. We can see the difference above for the user id 5, 10 & 15, The Popularity based model has recommended the same set of 5 products to both but Collaborative Filtering based model has recommended entire different list based on the user past purchase history.

2. Model-based Collaborative Filtering is a personalised recommender system, the recommendations are based on the past behavior of the user and it is not dependent on any additional information.

3. RMSE value of SVD Based Recommendation System using scipy library is 0.0812179913 and it is overfit Recommender model. Hence I tried with below options.

4. RMSE value of SVD Based Recommendation System using Surprise library is 1.7348 which proves that SVD model using scipy library is overfit model.

5. RMSE value of Recommendation System using KNNWithMeans is 1.1776 which is better than SVD models hence I could have tried cross validation and improve the accuracy.

<p style = "font-size:30px; color: #007580 "><strong> Thanks for reading 🙂</strong></p>